In [ ]:
import os
import sys
import pickle
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import networkx as nx
from torch_geometric.utils import from_networkx

# ----------------------------------------------------------------------
# 0. 프로젝트 루트 경로 추가 (필요한 경우)
# ----------------------------------------------------------------------
project_root = os.path.abspath(".")
if project_root not in sys.path:
    sys.path.append(project_root)

# ----------------------------------------------------------------------
# 1. TGN 모델 불러오기
# ----------------------------------------------------------------------
# 로컬에 복사한 TGN 코드는 model/tgn.py에 있습니다.
try:
    from model.tgn import TGN
except ImportError as e:
    raise ImportError("TGN 모델을 불러오지 못했습니다. model/tgn.py 파일의 경로를 확인하세요.") from e

# ----------------------------------------------------------------------
# 2. SnapshotSequenceDataset: 스냅샷 파일(.gpickle)로부터 동적 그래프 시퀀스 구성
# ----------------------------------------------------------------------
class SnapshotSequenceDataset(torch.utils.data.Dataset):
    """
    Snapshots 폴더 내의 모든 .gpickle 파일을 시간 순서대로 불러와,
    하나의 동적 그래프 시퀀스로 구성합니다.
    train 데이터는 label 정보를 포함하며, 
    test 데이터는 평가를 위해 ground truth label이 있으나, 스냅샷 생성 시에는 (drop_label 옵션으로)
    그래프 feature에서 label이 제거될 수 있습니다.
    """
    def __init__(self, snapshot_folder, label=None):
        self.snapshot_folder = snapshot_folder
        self.files = sorted([os.path.join(snapshot_folder, f) 
                             for f in os.listdir(snapshot_folder) if f.endswith('.gpickle')])
        self.label = label

    def __len__(self):
        # 전체 시퀀스를 하나의 샘플로 취급합니다.
        return 1

    def __getitem__(self, idx):
        sequence_x = []           # 각 스냅샷의 노드 feature 텐서
        sequence_edge_index = []  # 각 스냅샷의 edge_index 텐서
        for file in self.files:
            with open(file, 'rb') as f:
                G = pickle.load(f)
            # NetworkX 그래프를 PyG Data 객체로 변환
            data = from_networkx(G)
            # 노드 feature가 없으면, 각 노드의 degree를 feature로 사용 (1차원)
            if not hasattr(data, 'x') or data.x is None:
                deg = torch.tensor([val for (_, val) in G.degree()], dtype=torch.float).unsqueeze(1)
                data.x = deg
            sequence_x.append(data.x)
            sequence_edge_index.append(data.edge_index)
        return sequence_x, sequence_edge_index, self.label

# ----------------------------------------------------------------------
# 3. 스냅샷 생성 함수들
# ----------------------------------------------------------------------
def create_graph_from_snapshot(snapshot_df, drop_label=False):
    """
    snapshot_df의 각 행(패킷)을 하나의 통신으로 보고,
    'wlan.sa'와 'wlan.da'를 노드로 추가하며,
    나머지 feature (전체 feature 목록 중 'wlan.sa', 'wlan.da' 제외)를 엣지 속성으로 저장하는 그래프를 생성합니다.
    
    drop_label이 True이면, 각 패킷 정보 딕셔너리에서 'label' 키를 제거합니다.
    """
    G = nx.Graph()
    for idx, row in snapshot_df.iterrows():
        src = row.get('wlan.sa')
        dst = row.get('wlan.da')
        if pd.isna(src) or pd.isna(dst):
            continue
        packet_info = row.to_dict()
        packet_info.pop('wlan.sa', None)
        packet_info.pop('wlan.da', None)
        if drop_label:
            packet_info.pop('label', None)
        # Timestamp 타입이 있으면 float(timestamp)로 변환
        for key, value in packet_info.items():
            if isinstance(value, (np.datetime64, torch.Tensor)) or hasattr(value, 'timestamp'):
                try:
                    packet_info[key] = value.timestamp()
                except Exception:
                    packet_info[key] = float(value)
        if src not in G:
            G.add_node(src, type='device')
        if dst not in G:
            G.add_node(dst, type='device')
        if G.has_edge(src, dst):
            G[src][dst]['count'] += 1
            G[src][dst]['features'].append(packet_info)
        else:
            G.add_edge(src, dst, count=1, features=[packet_info])
    return G

def save_graph_snapshot(G, snapshot_time, attack_name, dataset_type="train"):
    """
    그래프 G를 pickle 파일로 저장합니다.
    파일명에 공격 이름과 snapshot_time(YYYYMMDD_HHMMSS)을 포함하며,
    dataset_type ("train" 또는 "test")에 따라 다른 폴더에 저장합니다.
    """
    time_str = snapshot_time.strftime('%Y%m%d_%H%M%S')
    file_name = f"{attack_name}_snapshot_{time_str}.gpickle"
    if dataset_type == "train":
        file_path = os.path.join(os.getcwd(), "Snapshots", "train", attack_name, file_name)
    else:
        file_path = os.path.join(os.getcwd(), "Snapshots", "test", attack_name, file_name)
    os.makedirs(os.path.dirname(file_path), exist_ok=True)
    with open(file_path, 'wb') as f:
        pickle.dump(G, f)
    print(f"Saved {dataset_type} snapshot: {file_path}")

def generate_dynamic_snapshots(df, attack_name, time_column='frame.time', interval='5min', drop_label=False):
    """
    DataFrame의 'frame.time' 열(에포크 초 값)을 기준으로,
    각 스냅샷을 처음부터 해당 시점까지의 누적 통신 데이터를 기반으로 생성합니다.
    생성된 스냅샷은 전체 통신 내역의 누적 그래프(NetworkX 객체)이며,
    drop_label이 True이면, 각 패킷 정보에서 'label'이 제거됩니다.
    """
    df[time_column] = pd.to_datetime(df[time_column], unit='s')
    df = df.sort_values(time_column)
    start_time = df[time_column].min()
    end_time = df[time_column].max()
    snapshots = {}
    current_time = start_time
    print(f"Creating cumulative snapshots for {attack_name} from {start_time} to {end_time} with interval {interval}.")
    while current_time < end_time:
        next_time = current_time + pd.Timedelta(interval)
        snapshot_df = df[df[time_column] < next_time]
        if not snapshot_df.empty:
            G_snapshot = create_graph_from_snapshot(snapshot_df, drop_label=drop_label)
            snapshots[current_time] = G_snapshot
            print(f"Cumulative Snapshot up to {next_time.strftime('%Y-%m-%d %H:%M:%S')} - {G_snapshot.number_of_nodes()} nodes, {G_snapshot.number_of_edges()} edges")
            dtype = "train" if not drop_label else "test"
            save_graph_snapshot(G_snapshot, current_time, attack_name, dataset_type=dtype)
        current_time = next_time
    print("All cumulative snapshots created.")
    return snapshots

# ----------------------------------------------------------------------
# 4. TGNClassifier 정의 (TGN 모델 래핑)
# ----------------------------------------------------------------------
class TGNClassifier(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, tgn_args):
        """
        in_channels: 입력 노드 feature 차원 (예: 1)
        hidden_channels: TGN 내부 hidden dimension
        out_channels: 분류할 클래스 수 (예: 14)
        tgn_args: TGN 초기화에 필요한 인자들을 담은 dict (neighbor_finder, node_features, edge_features, device 등)
        """
        super(TGNClassifier, self).__init__()
        self.tgn = TGN(**tgn_args)
        self.linear = nn.Linear(hidden_channels, out_channels)
    
    def forward(self, xs, edge_indices):
        h = None
        for x, edge_index in zip(xs, edge_indices):
            if edge_index.size(1) > 0:
                edge_attr = torch.ones((edge_index.size(1), 1), device=x.device)
                t_val = torch.zeros(edge_index.size(1), device=x.device)
            else:
                edge_attr = None
                t_val = None
            h = self.tgn(x, edge_index, edge_attr, t_val)
        h_mean = h.mean(dim=0)
        out = self.linear(h_mean)
        return out

# ----------------------------------------------------------------------
# 5. 학습 및 검증 설정
# ----------------------------------------------------------------------
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", device)

# 하이퍼파라미터 설정
in_channels = 1       # 예: 노드 feature가 degree이면 1
hidden_channels = 32
num_classes = 14      # 다중 분류: 14 클래스 (전처리 시 label_mapping 기준)

# tgn_args: TGN 초기화에 필요한 인자들 (예시 더미 값)
n_nodes = 1000
node_feat_dim = 10
edge_feat_dim = 5
# 더미 neighbor finder (실제 구현에서는 저장소 내 neighbor_finder 사용)
class DummyNeighborFinder:
    def __init__(self, n_nodes):
        self.n_nodes = n_nodes
    def find_before(self, node, timestamp, n_neighbors=20):
        return np.arange(min(n_neighbors, self.n_nodes))
dummy_neighbor_finder = DummyNeighborFinder(n_nodes)

tgn_args = {
    'neighbor_finder': dummy_neighbor_finder,
    'node_features': np.random.randn(n_nodes, node_feat_dim).astype(np.float32),
    'edge_features': np.random.randn(5000, edge_feat_dim).astype(np.float32),
    'device': device,
    'n_layers': 2,
    'n_heads': 2,
    'dropout': 0.1,
    'use_memory': True,
    'memory_update_at_start': True,
    'message_dimension': 100,
    'memory_dimension': 500,
    'embedding_module_type': "graph_attention",
    'message_function': "mlp",
    'mean_time_shift_src': 0,
    'std_time_shift_src': 1,
    'mean_time_shift_dst': 0,
    'std_time_shift_dst': 1,
    'n_neighbors': 20,
    'aggregator_type': "last",
    'memory_updater_type': "gru",
    'use_destination_embedding_in_message': False,
    'use_source_embedding_in_message': False,
    'dyrep': False
}

model = TGNClassifier(in_channels, hidden_channels, num_classes, tgn_args).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# 스냅샷 폴더 경로 설정 (예: Deauth 공격)
TRAIN_SNAPSHOT_FOLDER = os.path.join(os.getcwd(), "Snapshots", "train", "Deauth")
TEST_SNAPSHOT_FOLDER  = os.path.join(os.getcwd(), "Snapshots", "test", "Deauth")

# ----------------------------------------------------------------------
# 6. Snapshot 데이터셋 및 DataLoader 구성
# ----------------------------------------------------------------------
train_dataset = SnapshotSequenceDataset(TRAIN_SNAPSHOT_FOLDER, label=1)  # 예시: Deauth가 1번
test_dataset  = SnapshotSequenceDataset(TEST_SNAPSHOT_FOLDER, label=1)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=1, shuffle=False)
test_loader  = torch.utils.data.DataLoader(test_dataset, batch_size=1, shuffle=False)

# ----------------------------------------------------------------------
# 7. 학습 루프 (Train) - Train Loss, Test Loss, Test Accuracy 출력
# ----------------------------------------------------------------------
num_epochs = 50
for epoch in range(num_epochs):
    model.train()
    train_loss_epoch = 0
    for xs, edge_indices, label in train_loader:
        optimizer.zero_grad()
        output = model(xs[0].to(device), edge_indices[0].to(device))
        loss = criterion(output.unsqueeze(0), torch.tensor([label], dtype=torch.long, device=device))
        loss.backward()
        optimizer.step()
        train_loss_epoch += loss.item()
    train_loss_epoch /= len(train_loader)
    
    model.eval()
    test_loss_epoch = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for xs, edge_indices, label in test_loader:
            output = model(xs[0].to(device), edge_indices[0].to(device))
            loss = criterion(output.unsqueeze(0), torch.tensor([label], dtype=torch.long, device=device))
            test_loss_epoch += loss.item()
            pred = output.argmax(dim=-1)
            correct += (pred.item() == label)
            total += 1
    test_loss_epoch /= len(test_loader)
    test_acc = correct / total if total > 0 else 0
    print(f"Epoch {epoch+1}/{num_epochs}: Train Loss: {train_loss_epoch:.4f}, Test Loss: {test_loss_epoch:.4f}, Test Acc: {test_acc:.4f}")
    model.train()

# ----------------------------------------------------------------------
# 8. 최종 검증 (Test)
# ----------------------------------------------------------------------
model.eval()
with torch.no_grad():
    for xs, edge_indices, label in test_loader:
         output = model(xs[0].to(device), edge_indices[0].to(device))
         pred = output.argmax(dim=-1)
         print("Final Test Prediction:", pred.item())


Using device: cuda


TypeError: _unpickle_timestamp() takes exactly 3 positional arguments (4 given)